In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,16354.98,-0.12,-0.01,0.08,-0.00,-0.01,0.00,-0.00,-0.00
3,-0.12,3923.03,-0.02,0.01,-0.01,0.00,0.01,0.00,0.00
4,-0.01,-0.02,931.10,0.02,-0.00,-0.00,-0.00,-0.00,0.00
5,0.08,0.01,0.02,218.61,0.00,-0.00,0.00,0.00,0.00
6,-0.00,-0.01,-0.00,0.00,49.86,-0.00,-0.00,0.00,0.00
7,-0.01,0.00,-0.00,-0.00,-0.00,11.29,0.00,0.00,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.59,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.62,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,-0.00000,-0.00003,0.00002,-0.00001,-0.00001
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00007,0.00000,0.00002
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00003,1.00000,0.00001,-0.00003,0.00007,0.00001,0.00001
6,-0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00001,-0.00000,-0.00003,-0.00001,1.00000,0.00002,0.00000,0.00001
8,0.00002,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00003,0.00001
9,-0.00001,0.00000,-0.00001,0.00001,0.00003,0.00000,0.00003,1.00000,-0.00001
10,-0.00001,0.00002,0.00002,0.00001,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2112395036106749

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.97505483e+07 1.68388267e+06 9.29272325e+04 4.98130833e+03
 2.48883165e+02 1.19933682e+01 5.64758024e-01 2.48114243e-02
 1.19920167e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999469,-0.031759,-0.007075,-0.001650,-0.000377,-0.000086,-0.000019,-0.000005,-0.000001
1,0.031469,0.998819,-0.036104,-0.007954,-0.001812,-0.000409,-0.000092,-0.000022,-0.000005
2,0.008123,0.035486,0.998429,-0.041602,-0.008913,-0.002012,-0.000465,-0.000110,-0.000026
3,0.002211,0.009259,0.040779,0.997896,-0.048355,-0.010311,-0.002349,-0.000560,-0.000135
4,0.000605,0.002525,0.010645,0.047197,0.996937,-0.059935,-0.012914,-0.003059,-0.000739
5,0.000171,0.000709,0.002989,0.012762,0.058063,0.994781,-0.080791,-0.017950,-0.004318
6,0.000051,0.000212,0.000902,0.003825,0.016835,0.076888,0.988663,-0.124759,-0.027734
7,0.000017,0.000072,0.000305,0.001297,0.005690,0.025169,0.112858,0.963939,-0.239618
8,0.000009,0.000037,0.000155,0.000660,0.002897,0.012791,0.055751,0.234360,0.970461


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005309202394699897,
 0.0011809544883756296,
 0.0015710757440750012,
 0.002104177511772942,
 0.0030629668549114752,
 0.0052194155543705545,
 0.011336596456310133,
 0.0360608446915579,
 0.02953886510198056]